# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.4.1'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [9]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units=12, activation='tanh', input_shape = X_train.shape[1:]))

### Adding the second hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=18, activation='tanh'))

### Adding the output layer

In [12]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [13]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

EStop = EarlyStopping(monitor="val_accuracy", patience=10)

In [16]:
ann.fit(X_train, y_train,
        validation_data = (X_test, y_test),
        batch_size = 32, epochs = 100,
        callbacks = [EStop])

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5805 - accuracy: 0.6920 - val_loss: 0.4215 - val_accuracy: 0.8165
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4159 - accuracy: 0.8180 - val_loss: 0.4069 - val_accuracy: 0.8230
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4069 - accuracy: 0.8204 - val_loss: 0.3942 - val_accuracy: 0.8300
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4020 - accuracy: 0.8241 - val_loss: 0.3806 - val_accuracy: 0.8405
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3784 - accuracy: 0.8379 - val_loss: 0.3695 - val_accuracy: 0.8450
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3661 - accuracy: 0.8460 - val_loss: 0.3600 - val_accuracy: 0.8530
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3584 - accuracy: 0.8537 - val_loss: 0.3505 - val_accuracy: 0.8575

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [17]:
y_pred = ann.predict(X_test)
y_pred

array([[0.40778714],
       [0.3128594 ],
       [0.09065881],
       ...,
       [0.12820518],
       [0.15040705],
       [0.1872972 ]], dtype=float32)

In [18]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1498   97]
 [ 180  225]]


---
# Avaliando o Modelo

In [20]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [24]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units=12, kernel_initializer = 'uniform', activation='tanh', input_dim = 12))
    classifier.add(Dense(units=18, kernel_initializer = 'uniform', activation='tanh'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [25]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [26]:
print(mean,variance)

0.8486249923706055 0.011811672363365545


# Melhorando o modelo

In [39]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units=12, kernel_initializer = 'uniform', activation='tanh', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units=18, kernel_initializer = 'uniform', activation='tanh'))
    classifier.add(Dropout(rate = 0.2))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [40]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [41]:
print(mean,variance)

0.8423750042915344 0.012214877346631481


# Tuning the ANN

In [56]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=12, kernel_initializer = 'uniform', activation='tanh', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units=18, kernel_initializer = 'uniform', activation='tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [16, 32, 64],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    'callbacks': [EarlyStopping(patience=10, monitor="accuracy")]
}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [57]:
%%time

grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
450/450 [==============================] - 1s 657us/step - loss: 0.5547 - accuracy: 0.7989
Epoch 2/50
450/450 [==============================] - 0s 636us/step - loss: 0.4360 - accuracy: 0.8109
Epoch 3/50
450/450 [==============================] - 0s 647us/step - loss: 0.4302 - accuracy: 0.8128
Epoch 4/50
450/450 [==============================] - 0s 634us/step - loss: 0.4285 - accuracy: 0.8134
Epoch 5/50
450/450 [==============================] - 0s 669us/step - loss: 0.4218 - accuracy: 0.8199
Epoch 6/50
450/450 [==============================] - 0s 645us/step - loss: 0.4262 - accuracy: 0.8153
Epoch 7/50
450/450 [==============================] - 0s 657us/step - loss: 0.4263 - accuracy: 0.8118
Epoch 8/50
450/450 [==============================] - 0s 651us/step - loss: 0.4323 - accuracy: 0.8080
Epoch 9/50
450/450 [==============================] - 0s 682us/step - loss: 0.4234 - accuracy: 0.8170
Epoch 10/50
450/450 [==============================] - 0s 673us/step - loss: 0.430

c:\users\jose\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


450/450 [==============================] - 1s 619us/step - loss: 0.5533 - accuracy: 0.7966
Epoch 2/50
450/450 [==============================] - 0s 644us/step - loss: 0.4494 - accuracy: 0.7992
Epoch 3/50
450/450 [==============================] - 0s 720us/step - loss: 0.4214 - accuracy: 0.8188
Epoch 4/50
450/450 [==============================] - 0s 692us/step - loss: 0.4336 - accuracy: 0.8076
Epoch 5/50
450/450 [==============================] - 0s 629us/step - loss: 0.4327 - accuracy: 0.8143
Epoch 6/50
450/450 [==============================] - 0s 673us/step - loss: 0.4248 - accuracy: 0.8101
Epoch 7/50
450/450 [==============================] - 0s 657us/step - loss: 0.4307 - accuracy: 0.8082
Epoch 8/50
450/450 [==============================] - 0s 701us/step - loss: 0.4282 - accuracy: 0.8142
Epoch 9/50
450/450 [==============================] - 0s 648us/step - loss: 0.4171 - accuracy: 0.8193
Epoch 10/50
450/450 [==============================] - 0s 664us/step - loss: 0.4080 - accurac

c:\users\jose\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


450/450 [==============================] - 1s 626us/step - loss: 0.5539 - accuracy: 0.8039
Epoch 2/50
450/450 [==============================] - 0s 679us/step - loss: 0.4312 - accuracy: 0.8170
Epoch 3/50
450/450 [==============================] - 0s 634us/step - loss: 0.4379 - accuracy: 0.7999
Epoch 4/50
450/450 [==============================] - 0s 635us/step - loss: 0.4391 - accuracy: 0.8111
Epoch 5/50
450/450 [==============================] - 0s 639us/step - loss: 0.4397 - accuracy: 0.8072
Epoch 6/50
450/450 [==============================] - 0s 636us/step - loss: 0.4350 - accuracy: 0.8027
Epoch 7/50
450/450 [==============================] - 0s 654us/step - loss: 0.4285 - accuracy: 0.8111
Epoch 8/50
450/450 [==============================] - 0s 694us/step - loss: 0.4241 - accuracy: 0.8139
Epoch 9/50
450/450 [==============================] - 0s 637us/step - loss: 0.4282 - accuracy: 0.8099
Epoch 10/50
450/450 [==============================] - 0s 658us/step - loss: 0.4310 - accurac

...

113/113 [==============================] - 0s 839us/step - loss: 0.3490 - accuracy: 0.8564
Epoch 81/100
113/113 [==============================] - 0s 920us/step - loss: 0.3632 - accuracy: 0.8509
Epoch 82/100
113/113 [==============================] - 0s 839us/step - loss: 0.3601 - accuracy: 0.8507
Epoch 83/100
113/113 [==============================] - 0s 840us/step - loss: 0.3676 - accuracy: 0.8530
Epoch 84/100
113/113 [==============================] - 0s 866us/step - loss: 0.3608 - accuracy: 0.8529
Epoch 85/100
113/113 [==============================] - 0s 830us/step - loss: 0.3554 - accuracy: 0.8603
Epoch 86/100
113/113 [==============================] - 0s 857us/step - loss: 0.3565 - accuracy: 0.8523
Epoch 87/100
113/113 [==============================] - 0s 875us/step - loss: 0.3552 - accuracy: 0.8516
Epoch 88/100
113/113 [==============================] - 0s 839us/step - loss: 0.3513 - accuracy: 0.8588
Epoch 89/100
113/113 [==============================] - 0s 839us/step - loss:

c:\users\jose\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


113/113 [==============================] - 0s 848us/step - loss: 0.6573 - accuracy: 0.7750
Epoch 2/100
113/113 [==============================] - 0s 830us/step - loss: 0.4377 - accuracy: 0.8083
Epoch 3/100
113/113 [==============================] - 0s 883us/step - loss: 0.4348 - accuracy: 0.8040
Epoch 4/100
113/113 [==============================] - 0s 830us/step - loss: 0.4330 - accuracy: 0.8076
Epoch 5/100
113/113 [==============================] - 0s 838us/step - loss: 0.4329 - accuracy: 0.8102
Epoch 6/100
113/113 [==============================] - 0s 837us/step - loss: 0.4477 - accuracy: 0.8004
Epoch 7/100
113/113 [==============================] - 0s 875us/step - loss: 0.4271 - accuracy: 0.8096
Epoch 8/100
113/113 [==============================] - 0s 843us/step - loss: 0.4295 - accuracy: 0.8090
Epoch 9/100
113/113 [==============================] - 0s 875us/step - loss: 0.4346 - accuracy: 0.8086
Epoch 10/100
113/113 [==============================] - 0s 839us/step - loss: 0.4291 

c:\users\jose\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000001F8440F6F08>, as the constructor either does not set or modifies parameter callbacks

In [58]:
print('Best params:\n',grid_search.best_params_,'\nBest accuracy:\n',grid_search.best_score_)

Best params:
 {'batch_size': 16, 'callbacks': <tensorflow.python.keras.callbacks.EarlyStopping object at 0x000001F84890D1C8>, 'epochs': 100, 'optimizer': 'sgd'} 
Best accuracy:
 0.859


# Halving Grid Search

In [61]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=12, kernel_initializer = 'uniform', activation='tanh', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units=18, kernel_initializer = 'uniform', activation='tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [16, 32, 64],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop', 'sgd']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="accuracy", 
                                  n_jobs=-1, 
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10)

In [62]:
%%time
halving_cv = halving_cv.fit(X_train, y_train)

Epoch 1/100
500/500 [==============================] - 1s 804us/step - loss: 0.5486 - accuracy: 0.8014
Epoch 2/100
500/500 [==============================] - 0s 766us/step - loss: 0.4292 - accuracy: 0.8128
Epoch 3/100
500/500 [==============================] - 0s 802us/step - loss: 0.4341 - accuracy: 0.8121
Epoch 4/100
500/500 [==============================] - 0s 770us/step - loss: 0.4320 - accuracy: 0.8109
Epoch 5/100
500/500 [==============================] - 0s 774us/step - loss: 0.4319 - accuracy: 0.8128
Epoch 6/100
500/500 [==============================] - 0s 776us/step - loss: 0.4360 - accuracy: 0.8056
Epoch 7/100
500/500 [==============================] - 0s 774us/step - loss: 0.4254 - accuracy: 0.8131
Epoch 8/100
500/500 [==============================] - 0s 778us/step - loss: 0.4215 - accuracy: 0.8172
Epoch 9/100
500/500 [==============================] - 0s 780us/step - loss: 0.4055 - accuracy: 0.8269
Epoch 10/100
500/500 [==============================] - 0s 772us/step - l

500/500 [==============================] - 0s 635us/step - loss: 0.3511 - accuracy: 0.8565
Epoch 80/100
500/500 [==============================] - 0s 669us/step - loss: 0.3442 - accuracy: 0.8622
Epoch 81/100
500/500 [==============================] - 0s 639us/step - loss: 0.3452 - accuracy: 0.8586
Epoch 82/100
500/500 [==============================] - 0s 627us/step - loss: 0.3518 - accuracy: 0.8556
Epoch 83/100
500/500 [==============================] - 0s 691us/step - loss: 0.3424 - accuracy: 0.8591
Epoch 84/100
500/500 [==============================] - 0s 633us/step - loss: 0.3486 - accuracy: 0.8571
Epoch 85/100
500/500 [==============================] - 0s 629us/step - loss: 0.3621 - accuracy: 0.8525
Epoch 86/100
500/500 [==============================] - 0s 633us/step - loss: 0.3485 - accuracy: 0.8576
Epoch 87/100
500/500 [==============================] - 0s 631us/step - loss: 0.3563 - accuracy: 0.8545
Epoch 88/100
500/500 [==============================] - 0s 633us/step - loss:

In [63]:
print('Best params:\n',halving_cv.best_params_,'\nBest accuracy:\n',halving_cv.best_score_)

Best params:
 {'batch_size': 16, 'epochs': 100, 'optimizer': 'adam'} 
Best accuracy:
 0.8613266583229036


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)